In [347]:
#LIBRAIRIE
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
import ipympl
import ipywidgets as widgets
from ipywidgets import interactive, interact
%matplotlib widget
import plotly.express as px
import plotly.graph_objects as go

In [303]:
rea_all = pd.read_csv('../data_covid_clean.csv')

In [304]:
rea_all.drop(['deces', 'hospitalises', 'nouvelles_hospitalisations', 'nouvelles_reanimations', 'gueris'],1, inplace = True)

In [305]:
rea_all = rea_all.set_index('date')
rea_all = rea_all.sort_index()
rea_all = rea_all.loc['2020-03-18':]
rea_all = rea_all.loc[rea_all['source_nom']=="Santé publique France Data",:]
rea_all.drop(['source_nom'],1, inplace = True)
rea_all.head(50)

,maille_code,maille_nom,reanimation
date,,,
2020-03-18,DEP-62,Pas-de-Calais,9.0
2020-03-18,DEP-63,Puy-de-Dôme,2.0
2020-03-18,DEP-64,Pyrénées-Atlantiques,0.0
2020-03-18,DEP-65,Hautes-Pyrénées,0.0
2020-03-18,DEP-66,Pyrénées-Orientales,5.0
2020-03-18,DEP-70,Haute-Saône,4.0
2020-03-18,DEP-69,Rhône,8.0
2020-03-18,DEP-71,Saône-et-Loire,0.0
2020-03-18,DEP-72,Sarthe,0.0


In [306]:
rea_all.index = pd.to_datetime(rea_all.index)

In [307]:
rea_all.head()

,maille_code,maille_nom,reanimation
date,,,
2020-03-18,DEP-62,Pas-de-Calais,9.0
2020-03-18,DEP-63,Puy-de-Dôme,2.0
2020-03-18,DEP-64,Pyrénées-Atlantiques,0.0
2020-03-18,DEP-65,Hautes-Pyrénées,0.0
2020-03-18,DEP-66,Pyrénées-Orientales,5.0


In [308]:
DEP_code = pd.DataFrame(rea_all['maille_code'].unique())
for i in range(len(DEP_code)) :
    DEP_code.iat[i,0] = DEP_code.iat[i,0][4:]
DEP_code = DEP_code.loc[:,0].tolist()
DEP_name = pd.DataFrame(rea_all['maille_nom'].unique())
DEP_name = DEP_name.loc[:,0].tolist()
DEP = pd.concat([pd.DataFrame(DEP_code),pd.DataFrame(DEP_name)],1)
DEP = DEP.set_axis(['code', 'nom'], axis=1)
DEP = DEP.sort_values(by = 'code')
DEP = DEP.reset_index()
DEP = DEP.drop(['index'], 1)
DEP

,code,nom
0,01,Ain
1,02,Aisne
2,03,Allier
3,04,Alpes-de-Haute-Provence
4,05,Hautes-Alpes
...,...,...
96,971,Guadeloupe
97,972,Martinique
98,973,Guyane
99,974,La Réunion


In [309]:
def rea_DEP_nom(department):
    x = pd.DataFrame(rea_all.loc[rea_all['maille_nom']==f"{department}",'reanimation']).resample("1D").sum()
    x = x.rename(columns = {"reanimation": f"{department}"})
    return(x)

def rea_DEP_code(num_dep):
    x = pd.DataFrame(rea_all.loc[rea_all['maille_code']==f"DEP-{num_dep}",'reanimation']).resample("1D").sum()
    x = x.rename(columns = {"reanimation": f"{num_dep}"})
    return(x)

In [310]:
rea_all_code = pd.DataFrame()
for num_dep in DEP['code'].tolist() :
    rea_all_code = pd.concat([rea_all_code, rea_DEP_code(f'{num_dep}')], axis = 1)

rea_all_code = rea_all_code.astype(int)
rea_all_code = rea_all_code.reset_index()
rea_all_code.head()

rea_all_nom = pd.DataFrame()
for nom_dep in DEP['nom'].tolist() :
    rea_all_nom = pd.concat([rea_all_nom, rea_DEP_nom(f'{nom_dep}')], axis = 1)

rea_all_nom = rea_all_nom.astype(int)
rea_all_nom = rea_all_nom.reset_index()
rea_all_nom.head()

,date,Ain,Aisne,Allier,Alpes-de-Haute-Provence,Hautes-Alpes,Alpes-Maritimes,Ardèche,Ardennes,Ariège,...,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d'Oise,Guadeloupe,Martinique,Guyane,La Réunion,Mayotte
0,2020-03-18,0,10,0,1,1,1,1,0,1,...,16,48,33,32,29,0,5,0,2,0
1,2020-03-19,0,15,0,1,1,5,2,0,1,...,22,54,45,47,37,0,5,0,2,0
2,2020-03-20,0,19,0,1,1,7,3,1,1,...,28,74,62,60,51,0,5,0,2,0
3,2020-03-21,0,20,0,1,1,7,3,1,1,...,35,81,66,67,56,0,5,0,2,0
4,2020-03-22,1,21,0,1,1,8,3,1,1,...,47,95,75,78,66,6,5,0,2,0


In [311]:
fig = px.line(rea_all_nom, x="date", y=rea_all_nom.columns,
              hover_data={"date": "|%B %d, %Y"},
              title='intensive care beds occupied since 1st confinement in France by departments', height=500, width=800)
fig.update_xaxes(
    dtick='M1',
    tickformat="%d\n%b")
fig.show()

In [312]:
rea_all_Bretagne = pd.concat([rea_all_nom['date'], rea_all_nom['Finistère'], rea_all_nom["Côtes-d'Armor"], rea_all_nom["Morbihan"], rea_all_nom["Ille-et-Vilaine"]], axis=1)

rea_all_Normandie = pd.concat([rea_all_nom['date'], rea_all_nom['Manche'], rea_all_nom["Calvados"], rea_all_nom["Eure"], rea_all_nom["Orne"], rea_all_nom["Seine-Maritime"]], axis=1)

rea_all_Pays_de_la_Loire = pd.concat([rea_all_nom['date'], rea_all_nom['Mayenne'], rea_all_nom["Sarthe"], rea_all_nom["Vendée"], rea_all_nom["Loire-Atlantique"], rea_all_nom["Maine-et-Loire"]], axis=1)

rea_all_Centre_Val_de_Loire = pd.concat([rea_all_nom['date'], rea_all_nom['Cher'], rea_all_nom["Eure-et-Loir"], rea_all_nom["Indre"], rea_all_nom["Indre-et-Loire"], rea_all_nom["Loir-et-Cher"], rea_all_nom["Loiret"]], axis=1)

rea_all_Bourgogne_Franche_Comte = pd.concat([rea_all_nom['date'], rea_all_nom["Côte-d'Or"], rea_all_nom["Doubs"], rea_all_nom["Jura"], rea_all_nom["Nièvre"], rea_all_nom["Haute-Saône"], rea_all_nom["Saône-et-Loire"], rea_all_nom["Yonne"]], axis=1)

rea_all_Grand_Est = pd.concat([rea_all_nom['date'], rea_all_nom["Ardennes"], rea_all_nom["Aube"], rea_all_nom["Marne"], rea_all_nom["Haute-Marne"], rea_all_nom["Meurthe-et-Moselle"], rea_all_nom["Meuse"], rea_all_nom["Moselle"], rea_all_nom["Bas-Rhin"], rea_all_nom["Haut-Rhin"], rea_all_nom["Vosges"]], axis=1)

rea_all_Hauts_de_France = pd.concat([rea_all_nom['date'], rea_all_nom["Aisne"], rea_all_nom["Nord"], rea_all_nom["Oise"], rea_all_nom["Pas-de-Calais"], rea_all_nom["Somme"]], axis=1)

rea_all_Ile_de_France = pd.concat([rea_all_nom['date'], rea_all_nom["Paris"], rea_all_nom["Seine-et-Marne"], rea_all_nom["Yvelines"], rea_all_nom["Essonne"], rea_all_nom["Hauts-de-Seine"], rea_all_nom["Seine-Saint-Denis"], rea_all_nom["Val-de-Marne"], rea_all_nom["Val-d'Oise"]], axis=1)

rea_all_Nouvelle_Aquitaine = pd.concat([rea_all_nom['date'], rea_all_nom["Charente"], rea_all_nom["Charente-Maritime"], rea_all_nom["Corrèze"], rea_all_nom["Creuse"], rea_all_nom["Dordogne"], rea_all_nom["Gironde"], rea_all_nom["Landes"], rea_all_nom["Lot-et-Garonne"], rea_all_nom["Pyrénées-Atlantiques"], rea_all_nom["Deux-Sèvres"], rea_all_nom["Vienne"], rea_all_nom["Haute-Vienne"]], axis=1)

rea_all_Auvergne_Rhone_Alpes = pd.concat([rea_all_nom['date'], rea_all_nom["Ain"], rea_all_nom["Allier"], rea_all_nom["Ardèche"], rea_all_nom["Cantal"], rea_all_nom["Drôme"], rea_all_nom["Isère"], rea_all_nom["Loire"], rea_all_nom["Haute-Loire"], rea_all_nom["Puy-de-Dôme"], rea_all_nom["Rhône"], rea_all_nom["Savoie"], rea_all_nom["Haute-Savoie"]], axis=1)

rea_all_Occitanie = pd.concat([rea_all_nom['date'], rea_all_nom["Ariège"], rea_all_nom["Aude"], rea_all_nom["Aveyron"], rea_all_nom["Gard"], rea_all_nom["Haute-Garonne"], rea_all_nom["Gers"], rea_all_nom["Hérault"], rea_all_nom["Lot"], rea_all_nom["Lozère"], rea_all_nom["Hautes-Pyrénées"], rea_all_nom["Pyrénées-Orientales"], rea_all_nom["Tarn"], rea_all_nom["Tarn-et-Garonne"]], axis=1)

rea_all_Provence_Alpes_Côte_Azur = pd.concat([rea_all_nom['date'], rea_all_nom["Alpes-de-Haute-Provence"], rea_all_nom["Hautes-Alpes"], rea_all_nom["Alpes-Maritimes"], rea_all_nom["Bouches-du-Rhône"], rea_all_nom["Var"], rea_all_nom["Vaucluse"]], axis=1)

rea_all_Corse = pd.concat([rea_all_nom['date'], rea_all_nom["Corse-du-Sud"], rea_all_nom["Haute-Corse"]], axis=1)

rea_all_Outre_Mer = pd.concat([rea_all_nom['date'], rea_all_nom["Guadeloupe"], rea_all_nom["Martinique"], rea_all_nom["Guyane"], rea_all_nom["La Réunion"], rea_all_nom["Mayotte"]], axis=1)

In [313]:
rea_all_regions = {}
rea_all_regions['Bretagne'] = rea_all_Bretagne
rea_all_regions['Normandie'] = rea_all_Normandie
rea_all_regions['Pays de la Loire'] = rea_all_Pays_de_la_Loire
rea_all_regions['Centre-Val-de-Loire'] = rea_all_Centre_Val_de_Loire
rea_all_regions['Bourgogne-Franche-Comté'] = rea_all_Bourgogne_Franche_Comte
rea_all_regions['Grand-Est'] = rea_all_Grand_Est
rea_all_regions['Hauts-de-France'] = rea_all_Hauts_de_France
rea_all_regions['Île-de-France'] = rea_all_Ile_de_France
rea_all_regions['Nouvelle-Aquitaine'] = rea_all_Nouvelle_Aquitaine
rea_all_regions['Auvergne-Rhône-Alpes'] = rea_all_Auvergne_Rhone_Alpes
rea_all_regions['Occitanie'] = rea_all_Occitanie
rea_all_regions["Provence-Alpes-Côte-d'Azur"] = rea_all_Provences_Alpes_Côte_Azur
rea_all_regions['Corse'] = rea_all_Corse
rea_all_regions["Régions d'Outre Mer"] = rea_all_Outre_Mer

In [314]:
def rea_regions(region) :
    fig = px.line(rea_all_regions[f'{region}'], x="date", y=rea_all_regions[f'{region}'].columns,
              title=f'intensive care beds occupied since 1st confinement in {region}', height=500, width=800)
    fig.update_xaxes(
        dtick='M1',
        tickformat="%d\n%b")
    return(fig.show())

In [315]:
rea_regions('Occitanie')

In [316]:
Regions = [
    'Bretagne', 'Normandie', 'Pays de la Loire', 'Centre-Val-de-Loire', 'Bourgogne-Franche-Comté', 'Grand-Est', 'Hauts-de-France', 'Île-de-France', 'Nouvelle-Aquitaine', 'Auvergne-Rhône-Alpes', 'Occitanie', "Provence-Alpes-Côte-d'Azur", 'Corse', "Régions d'Outre Mer"
]

for name_region in Regions :
    rea_all_regions[f'{name_region}']['Total'] = np.sum(rea_all_regions[f'{name_region}'].iloc[:,1:],1)

In [317]:
rea_region_tot = pd.DataFrame()
for name_region in Regions :
    rea_region_tot = pd.concat([rea_region_tot, rea_all_regions[f'{name_region}']['Total']], 1)

In [318]:
rea_region_tot.columns = Regions
rea_region_tot.insert(0, 'date', rea_all_nom['date'])
rea_region_tot.head(15)

,date,Bretagne,Normandie,Pays de la Loire,Centre-Val-de-Loire,Bourgogne-Franche-Comté,Grand-Est,Hauts-de-France,Île-de-France,Nouvelle-Aquitaine,Auvergne-Rhône-Alpes,Occitanie,Provence-Alpes-Côte-d'Azur,Corse,Régions d'Outre Mer
0,2020-03-18,23,4,4,5,24,204,87,294,12,35,39,48,0,7
1,2020-03-19,26,6,11,10,38,241,105,360,11,79,55,88,0,7
2,2020-03-20,25,13,17,13,62,294,136,484,15,87,66,110,14,7
3,2020-03-21,24,21,21,14,68,332,144,545,28,88,70,132,14,7
4,2020-03-22,31,25,25,21,69,370,151,640,40,106,78,156,13,13
5,2020-03-23,30,33,35,38,79,432,187,787,47,157,101,224,9,13
6,2020-03-24,30,40,43,44,99,504,208,973,69,190,125,266,9,14
7,2020-03-25,38,56,49,58,123,575,247,1114,75,258,147,286,9,16
8,2020-03-26,45,75,59,69,136,636,271,1269,96,290,177,328,13,21
9,2020-03-27,49,93,84,80,156,699,302,1389,116,344,187,372,13,30


In [319]:
fig = px.line(rea_region_tot, x="date", y=rea_region_tot.columns,
              title=f'intensive care beds occupied since 1st confinement by regions', height=500, width=800)
fig.update_xaxes(
        dtick='M1',
        tickformat="%b")
fig.show()

In [320]:
rea_region_tot['Total'] = np.sum(rea_region_tot.iloc[:,1:],1)

In [321]:
rea_region_tot.head(2)

,date,Bretagne,Normandie,Pays de la Loire,Centre-Val-de-Loire,Bourgogne-Franche-Comté,Grand-Est,Hauts-de-France,Île-de-France,Nouvelle-Aquitaine,Auvergne-Rhône-Alpes,Occitanie,Provence-Alpes-Côte-d'Azur,Corse,Régions d'Outre Mer,Total
0,2020-03-18,23,4,4,5,24,204,87,294,12,35,39,48,0,7,786
1,2020-03-19,26,6,11,10,38,241,105,360,11,79,55,88,0,7,1037


In [322]:
fig = px.bar(rea_region_tot, x='date', y='Total',
    title=f'patients in reanimation flux in France during Covid19 crisis', color='Total', labels={'Total':'Number of patients'}, height=500, width=800)
fig.show()

In [323]:
rea_all_period_by_region = pd.DataFrame(np.sum(rea_region_tot.iloc[:,1:15],0))
rea_all_period_by_region.columns = ['Total number']
rea_all_period_by_region

,Total number
Bretagne,14295
Normandie,26923
Pays de la Loire,22010
Centre-Val-de-Loire,28779
Bourgogne-Franche-Comté,35215
Grand-Est,89149
Hauts-de-France,85152
Île-de-France,268341
Nouvelle-Aquitaine,39727
Auvergne-Rhône-Alpes,112575


In [324]:
fig = px.pie(rea_all_period_by_region, values='Total number', names=rea_all_period_by_region.index, title='Repartition of patients in reanimation by region since the begining of Covid19 crisis', color_discrete_sequence=px.colors.sequential.RdBu, height=500, width=800)
fig.show()

In [335]:
beds_in_rea_dep = pd.read_csv('../bed_rea_dep.csv', delimiter=';')
beds_in_rea_reg = pd.read_csv('../bed_rea_reg.csv', delimiter=';')

In [339]:
beds_in_rea_dep.head(-25)

,NUM_DEP,DEP,Lits rea dispo
0,01,Ain,22
1,02,Aisne,34
2,03,Allier,24
3,04,Alpes-de-Haute-Provence,6
4,05,Hautes-Alpes,8
...,...,...,...
71,71,Saône-et-Loire,26
72,72,Sarthe,21
73,73,Savoie,18
74,74,Haute-Savoie,34


In [328]:
beds_in_rea_reg = beds_in_rea_reg.rename(columns={'Unnamed: 0' : 'Région'})
beds_in_rea_reg = beds_in_rea_reg.drop(['CHR', 'Autres'], 1)

In [329]:
beds_in_rea_reg = beds_in_rea_reg.loc[:13]
beds_in_rea_reg = beds_in_rea_reg.rename(columns={'Confondu' : 'Nombre de lits'})
beds_in_rea_reg = beds_in_rea_reg.reindex(index=[2,8,11,3,1,5,6,7,9,0,10,12,4,13])
beds_in_rea_reg = beds_in_rea_reg.reset_index()
beds_in_rea_reg = beds_in_rea_reg.drop(['index'],1)


In [330]:
beds_in_rea_reg = beds_in_rea_reg.set_index('Région')

In [340]:
beds_in_rea_dep = beds_in_rea_dep.drop(['NUM_DEP'], 1)
beds_in_rea_dep = beds_in_rea_dep.set_index('DEP')

In [341]:
beds_in_rea_dep

,Lits rea dispo
DEP,
Ain,22
Aisne,34
Allier,24
Alpes-de-Haute-Provence,6
Hautes-Alpes,8
...,...
Guadeloupe,33
Martinique,29
Guyane,13


In [332]:
rea_reg_prop = rea_region_tot.drop(['Total'],1)
rea_reg_prop

for col in Regions :
    for row in range(len(rea_reg_prop)) :
        rea_reg_prop.loc[row, f'{col}'] = rea_reg_prop.loc[row, f'{col}']/int(beds_in_rea_reg.loc[f'{col}','Nombre de lits'])


In [343]:
rea_dep_prop = rea_all_nom.copy()

for col in beds_in_rea_dep.index :
    for row in range(len(rea_dep_prop)) :
        rea_dep_prop.loc[row, f'{col}'] = rea_dep_prop.loc[row, f'{col}']/int(beds_in_rea_dep.loc[f'{col}','Lits rea dispo'])

In [334]:
rea_reg_prop

,date,Bretagne,Normandie,Pays de la Loire,Centre-Val-de-Loire,Bourgogne-Franche-Comté,Grand-Est,Hauts-de-France,Île-de-France,Nouvelle-Aquitaine,Auvergne-Rhône-Alpes,Occitanie,Provence-Alpes-Côte-d'Azur,Corse,Régions d'Outre Mer
0,2020-03-18,0.134503,0.015504,0.020202,0.026316,0.118227,0.403960,0.183544,0.240000,0.027088,0.057003,0.080913,0.097959,0.000000,0.043210
1,2020-03-19,0.152047,0.023256,0.055556,0.052632,0.187192,0.477228,0.221519,0.293878,0.024831,0.128664,0.114108,0.179592,0.000000,0.043210
2,2020-03-20,0.146199,0.050388,0.085859,0.068421,0.305419,0.582178,0.286920,0.395102,0.033860,0.141694,0.136929,0.224490,0.777778,0.043210
3,2020-03-21,0.140351,0.081395,0.106061,0.073684,0.334975,0.657426,0.303797,0.444898,0.063205,0.143322,0.145228,0.269388,0.777778,0.043210
4,2020-03-22,0.181287,0.096899,0.126263,0.110526,0.339901,0.732673,0.318565,0.522449,0.090293,0.172638,0.161826,0.318367,0.722222,0.080247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2021-03-12,0.350877,0.608527,0.479798,0.757895,0.704433,0.651485,1.033755,0.894694,0.424379,0.718241,0.564315,1.979592,0.555556,0.697531
360,2021-03-13,0.380117,0.612403,0.479798,0.747368,0.714286,0.669307,1.050633,0.897959,0.428894,0.724756,0.541494,2.000000,0.666667,0.753086
361,2021-03-14,0.380117,0.651163,0.449495,0.763158,0.709360,0.683168,1.065401,0.925714,0.442438,0.713355,0.549793,2.016327,0.666667,0.734568
362,2021-03-15,0.403509,0.635659,0.459596,0.752632,0.729064,0.706931,1.082278,0.951837,0.453725,0.724756,0.574689,2.093878,0.722222,0.679012


In [344]:
rea_dep_prop

,date,Ain,Aisne,Allier,Alpes-de-Haute-Provence,Hautes-Alpes,Alpes-Maritimes,Ardèche,Ardennes,Ariège,...,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d'Oise,Guadeloupe,Martinique,Guyane,La Réunion,Mayotte
0,2020-03-18,0.000000,0.294118,0.000000,0.166667,0.125,0.010309,0.125,0.000000,0.125,...,0.164948,0.228571,0.347368,0.197531,0.557692,0.000000,0.172414,0.000000,0.026667,0.000000
1,2020-03-19,0.000000,0.441176,0.000000,0.166667,0.125,0.051546,0.250,0.000000,0.125,...,0.226804,0.257143,0.473684,0.290123,0.711538,0.000000,0.172414,0.000000,0.026667,0.000000
2,2020-03-20,0.000000,0.558824,0.000000,0.166667,0.125,0.072165,0.375,0.066667,0.125,...,0.288660,0.352381,0.652632,0.370370,0.980769,0.000000,0.172414,0.000000,0.026667,0.000000
3,2020-03-21,0.000000,0.588235,0.000000,0.166667,0.125,0.072165,0.375,0.066667,0.125,...,0.360825,0.385714,0.694737,0.413580,1.076923,0.000000,0.172414,0.000000,0.026667,0.000000
4,2020-03-22,0.045455,0.617647,0.000000,0.166667,0.125,0.082474,0.375,0.066667,0.125,...,0.484536,0.452381,0.789474,0.481481,1.269231,0.181818,0.172414,0.000000,0.026667,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2021-03-12,0.727273,1.088235,0.583333,1.500000,1.125,1.206186,1.250,0.666667,1.000,...,1.123711,0.766667,1.452632,0.820988,0.884615,0.212121,0.275862,0.461538,0.893333,2.083333
360,2021-03-13,0.863636,1.088235,0.625000,1.166667,1.125,1.216495,1.375,0.733333,0.750,...,1.051546,0.766667,1.442105,0.802469,0.942308,0.272727,0.344828,0.538462,0.946667,2.083333
361,2021-03-14,0.863636,1.147059,0.541667,1.000000,1.000,1.237113,1.375,0.800000,0.750,...,1.113402,0.771429,1.494737,0.820988,1.019231,0.303030,0.310345,0.538462,0.906667,2.083333
362,2021-03-15,0.909091,1.205882,0.500000,1.166667,1.000,1.329897,1.500,0.800000,0.750,...,1.144330,0.833333,1.452632,0.845679,1.096154,0.363636,0.241379,0.538462,0.813333,1.916667


In [370]:
fig = px.imshow(rea_reg_prop.iloc[:,1:], y=rea_reg_prop['date'], title='Regional saturation in reanimation service', height=700, width=800)
fig.show()

In [369]:
fig = px.imshow(rea_dep_prop.iloc[:,1:], y=rea_dep_prop['date'], title='Departmental saturation in reanimation service', height=1000, width=3000)
fig.show()